# Individual glacier analysis 1

This notebook will walk you through steps to read in and organize velocity data and clip it to the extent of a single glacier. The tools we will use include **xarray**, **rioxarray** and **geopandas**. 

To clip its_live data to the extent of a single glacier we will use a vector dataset of glacier outlines, the [Randolph Glacier Inventory](https://nsidc.org/data/nsidc-0770). These aren't cloud-hosted currently so you will need to download the data to your local machine. 

**Learning goals**
come back and finish these, feel like this notebook has alot, is pretty disorganized.. </br>
using xarray to read zarr data from s3 bucket
- **`rio.clip()`** to clip raster by vector
- viewing CRS, reprojecting and writing CRS data for various objects
- dataset.where()
- dataset.sel() using multiple conditions
- groupby



First, lets install the python libraries that were listed on the [Software](software.ipynb) page:

In [ ]:
import geopandas as gpd
import os
import numpy as np
import xarray as xr
import rioxarray as rxr
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from shapely.geometry import Polygon
from shapely.geometry import Point
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy
import cartopy.feature as cfeature
import json
import urllib.request
from skimage.morphology import skeletonize
import pandas as pd
import seaborn as sns 
from matplotlib import pyplot as plt
%config InlineBackend.figure_format='retina'


## Reading in ITS_LIVE data

We will use some of the functions we defiend in the data access notebook to read in data here. 

In [ ]:
import itslivetools

First, let's read in the catalog again:  

In [ ]:
with urllib.request.urlopen('https://its-live-data.s3.amazonaws.com/datacubes/catalog_v02.json') as url_catalog:
    itslive_catalog = json.loads(url_catalog.read().decode())
itslive_catalog.keys()

Take a look at a single catalog entry:

Use the function below to find the url that corresponds to the zarr datacube for a specific point:

The `read_in_s3()` function will read in a xarray dataset from a url to a zarr datacube when we're ready:

I started with `chunk_size='auto'` but ran into issues. more about choosing good chunk sizes [here](https://blog.dask.org/2021/11/02/choosing-dask-chunk-sizes). 

In [ ]:
url = itslivetools.find_granule_by_point(itslive_catalog, [84.56, 28.54])
url

In [ ]:
dc = itslivetools.read_in_s3(url[0])
dc

We are reading this in as a dask array. Let's take a look at the chunk sizes:

**NOTE**: chunksizes shows the largest chunk size. chunks shows the sizes of all chunks along all dims, better if you have irregular chunks

In [ ]:
dc.chunksizes

In [ ]:
dc.chunks

Setting the dask chunksize to `auto` at the `xr.open_dataset()` step will use chunk sizes that most closely resemble the structure of the underlying data. To avoid imposing a chunk size that isn't a good fit for the data, avoid re-chunking until we have selected a subset of our area of interest from the larger dataset 

Check CRS of xr object: 

In [ ]:
dc.mapping

Let's take a look at the time dimension (`mid_date` here). To start with we'll just print the first 10 values:

In [ ]:
for element in range(10):
    
    print(dc.mid_date[element].data)

Weird, it doesn't look like the time dimension is in chronological order, let's fix that: 

In [ ]:
dc_timesorted = dc.sortby(dc['mid_date'])
dc_timesorted

When we read in the zarr datacube as a `xr.Dataset` we set the chunk sizes to `auto`. When we try to sort along the `mid_date` dimension this seems to become a problem and we get the warning above. 

At first it makes sens to follow the instructions in the warning message to avoid creating large chunks, but this creates some issues. If you want, turn the cell below to `code` and run it, you can see that this re-chunks the time dimension which isn't something that we want 

In [ ]:
import dask
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    dc_timesorted_false = dc.sortby(dc['mid_date'])
    dc_timesorted_false

In [ ]:
dc_timesorted

In [ ]:
for element in range(10):
    
    print(dc_timesorted.mid_date[element].data)

## Read in vector data 

We are going to read in RGI region **15 (SouthAsiaEast)**. RGI data is downloaded in lat/lon coordinates. We will project it to match the CRS of the ITS_LIVE dataset and then select an individual glacier to begin our analysis.

In [ ]:
se_asia = gpd.read_file('/Users/emarshall/Desktop/siparcs/data/nsidc0770_15.rgi60.SouthAsiaEast/15_rgi60_SouthAsiaEast.shp')
se_asia.head(3)

In [ ]:
#project rgi data to match itslive
se_asia_prj = se_asia.to_crs('EPSG:32645') #we know the epsg from looking at the 'spatial epsg' attr of the mapping var of the dc object
se_asia_prj.head(3)

## Crop RGI to ITS_LIVE extent
- use get_bbox_single() from access nb but no plotting (above)

In [ ]:
#first, get vector bbox of itslive

bbox_dc = itslivetools.get_bbox_single(dc)
bbox_dc['geometry']

In [ ]:
#project from latlon to local utm 
bbox_dc = bbox_dc.to_crs('EPSG:32645')
bbox_dc

In [ ]:
#subset rgi to bounds 
se_asia_subset = gpd.clip(se_asia_prj, bbox_dc)
se_asia_subset
se_asia_subset.explore()

In [ ]:
sample_glacier_vec = se_asia_subset.loc[se_asia_subset['RGIId'] == 'RGI60-15.04714']
sample_glacier_vec

### Clip ITS_LIVE dataset to individual glacier extent

First, we need to use rio.write_crs() to assign a CRS to the itslive object. If we don't do that first the `rio.clip()` command will produce an error
*Note*: it looks like you can only run write_crs() once, because it switches mapping from being a `data_var` to a `coord` so if you run it again it will produce a key error looking for a var that doesnt' exist

In [ ]:
dc_timesorted = dc_timesorted.rio.write_crs(f"epsg:{dc_timesorted.mapping.attrs['spatial_epsg']}", inplace=True)

In [ ]:
%%time

sample_glacier_raster = dc_timesorted.rio.clip(sample_glacier_vec.geometry, sample_glacier_vec.crs)

Take a look at the clipped object:

In [ ]:
sample_glacier_raster

Let's take a look at the clipped raster alongside the vector outline. To start with and for the sake of easy visualizing we will take the mean of the magnitude of velocity variable along the `mid_date` dimension:

In [ ]:
fig, ax = plt.subplots(figsize = (15,9))
sample_glacier_vec.plot(ax=ax, facecolor='none', edgecolor='red');
sample_glacier_raster.v.mean(dim=['mid_date']).plot(ax=ax);



Now let's take a look at the x and y components of velocity, again averaging over time:

In [ ]:
fig, axs = plt.subplots(ncols =2, figsize=(17,7))

sample_glacier_raster.vx.mean(dim='mid_date').plot(ax=axs[0]);
sample_glacier_raster.vy.mean(dim='mid_date').plot(ax=axs[1]);


In [ ]:
sample_glacier_raster.v_error.mean(dim=['mid_date']).plot();

## Exploring ITS_LIVE data

ITS_LIVE data cubes come with many (53!) variables that carry information about the estimated surface velocities and the satellite images that were used to generate the surface velocity estimates. We won't examine all of this information here but let's look at a litte bit.

To start with, let's look at the satellite imagery used to generate the velocity data.

We see that we have two `data_vars` that indicate which sensor that each image in the image pair at a certain time step comes from:

In [ ]:
sample_glacier_raster.satellite_img1.data.compute()

In [ ]:
sample_glacier_raster.satellite_img2

The `satellite_img1` and `satellite_img2` variables are 1-dimensional numpy arrays corresponding to the length of the `mid_date` dimension of the data cube. You can see that each element of the array is a string corresponding to a different satellite:
    `1A` = Sentinel 1A, `1B` = Sentinel 1B, `2A` = Sentinel 2A
    `2B` = Sentinel 2B, `8.` = Landsat8 and `9.` = Landsat9
    
Let's re-arrange these string arrays into a format that is easier to work with.

First, we'll make a set of all the different string values in the satellite image variables:

## Examining velocity data from each satellite in `ITS_LIVE` dataset

What if we only wanted to look at the velocity estimates from landat8?

In [ ]:
l8_data = sample_glacier_raster.where(sample_glacier_raster['satellite_img1'] == '8.', drop=True)
l8_data

`dataset.where()` at first seems appropriate to use for kind of operation but there's actually an easier way. Because we are selecting along a single dimension (`mid_date`), we can use xarray's `.sel()` method instead. This is more efficient and integrates with `dask` arrays more smoothly.

In [ ]:
l8_condition = sample_glacier_raster.satellite_img1.isin('8.')
l8_subset = sample_glacier_raster.sel(mid_date=l8_condition)
l8_subset

We can see that we are looking at roughly a third of the original time steps. Let's take a look at the average speeds of the Landsat8-derived velocities:

In [ ]:
l8_subset.v.mean(dim='mid_date').plot();

What about Landsat9?

In [ ]:
l9_condition = sample_glacier_raster.satellite_img1.isin('9.')

l9_subset = sample_glacier_raster.sel(mid_date=l9_condition)
l9_subset

Only 45 time steps have data from Landsat9, this makes sense because Landsat9 was just launched recently

In [ ]:
l9_subset.v.mean(dim='mid_date').plot();

Let's look at Sentinel 1 data. Note here we are selecting for 2 values instead of 1: 

In [ ]:
s1_condition = sample_glacier_raster.satellite_img1.isin(['1A','1B'])
s1_subset = sample_glacier_raster.sel(mid_date = s1_condition)
s1_subset

In [ ]:
s1_subset.v.mean(dim='mid_date').plot();

In [ ]:
s2_condition = sample_glacier_raster.satellite_img1.isin(['2A','2B'])
s2_subset = sample_glacier_raster.sel(mid_date=s2_condition)
s2_subset

In [ ]:
s2_subset.v.mean(dim='mid_date').plot();

## Checking coverage along a dimension
It would be nice to be able to scan/visualize and observe coverage of a variable along a dimension

Attempting that below...

First need to make a mask that will tell us all the possible 'valid' pixels. ie pixels over ice v. rock. Some versions of ITS_LIVE have a variable that indicates surface type that would be handy for this but the datacubes don't. Start by taking the mean of velocity pixels along the time dimension, take all pixels in this array that aren't NaN as ice pixels. 

In [ ]:
valid_pixels = sample_glacier_raster.v.count(dim=['x','y']).compute()
valid_pix_max = valid_pixels.max().data

cov = valid_pixels/valid_pix_max

sample_glacier_raster['cov'] = cov
sample_glacier_raster

In [ ]:
#how many time steps are duplicates?, there are 16872 unique vals in mid_dates
np.unique(sample_glacier_raster['mid_date'].data).shape

In [ ]:
fig, ax = plt.subplots(figsize=(30,3))
sample_glacier_raster.cov.plot(ax=ax, linestyle='None',marker = 'x')

In [ ]:
sample_glacier_raster.groupby('satellite_img1').mean()

In [ ]:
sample_glacier_raster.cov.groupby(sample_glacier_raster.satellite_img1)

In [ ]:
sample_glacier_raster.groupby('mid_date')

In [ ]:
import flox.xarray

In [ ]:
sample_glacier_raster.cov

In [ ]:
coverage = flox.xarray.xarray_reduce(
    sample_glacier_raster.cov,
    sample_glacier_raster.satellite_img1.compute(),
    sample_glacier_raster.mid_date,
    func="mean",
    fill_value=0,
)

In [ ]:
plt.pcolormesh(coverage.mid_date, coverage.satellite_img1, coverage, cmap='viridis')
plt.colorbar()

## Seasonal mean velocities for each year with groupby

In [ ]:
year_gb = sample_glacier_raster.groupby(sample_glacier_raster.mid_date.dt.year)
year_gb

In [ ]:
coverage = flox.xarray.xarray_reduce(
    sample_glacier_raster.cov,
    sample_glacier_raster.satellite_img1.compute(),
    sample_glacier_raster.mid_date,
    func="mean",
    fill_value=0,
)

In [ ]:
yr_season_mean_v = flox.xarray.xarray_reduce(
    sample_glacier_raster.v,
    sample_glacier_raster.mid_date.dt.year,
    sample_glacier_raster.mid_date.dt.season,
    func = 'mean',
)

In [ ]:
yr_season_mean.compute()

In [ ]:
yr_season_mean

In [ ]:
fg_yr = yr_season_mean.plot(
        col='season',
        row = 'year',)

## Seasonal mean velocities with groupby

In [ ]:
#first define the function we'll apply to each group
def middate_mean(a):
    return a.mean(dim='mid_date')


In [ ]:
seasons_gb = sample_glacier_raster.groupby(sample_glacier_raster.mid_date.dt.season).map(middate_mean)
#add attrs to gb object
seasons_gb.attrs = sample_glacier_raster.attrs #why didn't that work?
seasons_gb

In [ ]:
fg = seasons_gb.v.plot(
    col='season',
    vmax = 150
);